In [1]:
import os
import shutil
import json
import logging

In [2]:
directory=('../data/train/wav')
files=[]
target=('../data/alldata')

In [3]:
for folders in os.listdir(directory):
        #try cacth erros
        try:
        
            #accessing subfolders inside train/wav directory
            subfolder = os.path.join(directory,folders)
            
            #looping through all contents in the subfolder
            for wavz in os.listdir(subfolder):

                files.append(wavz)
                finalpath = os.path.join(subfolder,wavz)
                #copying files to new audio_path
                shutil.copy(finalpath, target)
        except FileNotFoundError:
            print(" !!! Error File Not Found  !!!!! \n")
            print(" !!! Program Failed !!!!! \n")
            logging.error(" !!! Error Program Failed !!!!! \n")
        except Exception as e:
            print(" !!! Error !!!!! \n")
            

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data/train/wav'

In [ ]:
name_to_text={}

In [ ]:
filename=('../data/train/text')
with open (filename, encoding="utf-8")as f:
        try:
            #open the txt file containnig file and matching text file
            
            f.readline()
            for line in f:
                #split first half to be the value and second part is the key/text
                
                name=line.split("\t")[-1]
                name=name.rstrip()
                file=line.split("\t")[0]
                file=file+".wav"
                length=len(name)         
                name_to_text[file]=[name,length]
            
        except Exception as e:
            print (" !!!! Error !!!! ")
            print (" !!!! The system raised an exception {} !!!!!".format(e.__class__))

with open("../data/meta_data.json", "w") as outfile: 
        json.dump(name_to_text, outfile)
        
        
#function to get csv file with names 
def get_file():
    #we convert the json file we got from the previous function into a dataframe
    try:
        print(" ========= Converting dictionary to dataframe ================= \n")
        logging.info("======= Converting dictionary to dataframe ================= \n")

        data = pd.DataFrame.from_dict(name_to_text, orient ='index')
        data=data.reset_index()
        print("======= Creating columns ================= \n")
        data.columns=['wav_file','text','length']
        
        data.to_csv("../data/merged_data.csv",index=False)  
    except Exception as e:
        print (" !!!! Error !!!! ")
        print (" !!!! The system raised an exception {} !!!!!".format(e.__class__))

In [ ]:
#function
#before using this function kindly change your file paths for it to work


def make_stereo(audio_path):
    #this function converts mono audio channels into stereo channels 
#     logging.info(" ============ Conerting audio sample from mono to stereo ================= ")
    print("======= Mono to stereo audio conversion")
    ifile = wave.open(audio_path)
    #log the info on adio files
#     logging.info(ifile.getparams())
    print (ifile.getparams())
    # (1, 2, 44100, 2013900, 'NONE', 'not compressed')
    (nchannels, sampwidth, framerate, nframes, comptype, compname) = ifile.getparams()
    assert (comptype == 'NONE')  # Compressed not supported yet
    array_type = {1:'B', 2: 'h', 4: 'l'}[sampwidth]
    print(" ======= Calculting left channel type =====")
    left_channel = array.array(array_type, ifile.readframes(nframes))[::nchannels]
    ifile.close()

    #convert the number of channels to 2
    print("====== converting channels ======= ")
    stereo = 2 * left_channel
    stereo[0::2] = stereo[1::2] = left_channel
    #overwrite the wav file making it a stereo file
    print("====== overwriting wav file ======= ")
    ofile = wave.open(audio_path, 'w')
    ofile.setparams((2, sampwidth, framerate, nframes, comptype, compname))
    ofile.writeframes(stereo.tostring())
    ofile.close()

In [ ]:
data['wav_file']=data['wav_file'].apply(lambda x: make_stereo(x))